In [1]:
import sys
import pickle
import glob
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import warnings

warnings.filterwarnings(action="once")

if '/home/anna/BioSimSpace/python' not in sys.path:
    sys.path.insert(1, '/home/anna/BioSimSpace/python')
import BioSimSpace as BSS

# if '/home/anna/Documents/cinnabar' not in sys.path:
#     sys.path.insert(1, '/home/anna/Documents/cinnabar')
# import cinnabar

# print(cinnabar.__file__)

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *

# import BioSimSpace.Sandpit.Exscientia as BSS
# import BioSimSpace as BSS


BSS.__file__

import py3Dmol as _py3Dmol
from rdkit import Chem as _Chem
import tempfile as _tempfile

/home/anna/mambaforge/envs/openbiosim-dev/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


INFO:rdkit:Enabling RDKit 2023.03.2 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/home/anna/BioSimSpace/python/BioSimSpace/FreeEnergy/_relative.py:49: UserWarning: can't import pyarrow.parquet, cannot analyse somd2 files.
  _warnings.warn("can't import pyarrow.parquet, cannot analyse somd2 files.")
/home/anna/BioSimSpace/python/BioSimSpace/Parameters/_Protocol/_amber.py:57: ResourceWarning: unclosed file <_io.TextIOWrapper name='/dev/null' mode='w' encoding='UTF-8'>
  _sys.stderr = _sys.__stderr__
/home/anna/BioSimSpace/python/BioSimSpace/Parameters/_Protocol/_openforcefield.py:88: ResourceWarning: unclosed file <_io.TextIOWrapper name='/dev/null' mode='w' encoding='UTF-8'>
  _sys.stderr = _sys.__stderr__


adding code to the pythonpath...


In [2]:
import rdkit
rdkit.__version__

'2023.03.2'

In [3]:
def draw_labelled_molecule(lig_0):
    # Create a temporary working directory and store the directory name.
    tmp_dir = _tempfile.TemporaryDirectory()
    work_dir = tmp_dir.name

    BSS.IO.saveMolecules(work_dir + "/molecule0", lig_0, "PDB")
    # Load the molecules into RDKit.
    rdmol0 = _Chem.MolFromPDBFile(
        work_dir + "/molecule0.pdb", sanitize=False, removeHs=False
    )

    # Set grid view properties.
    pixels = 900
    width = pixels / 2
    height = pixels

    # Create the view.
    view = _py3Dmol.view(linked=False, width=width, height=height)
    style = {"stick": {"colorscheme": "grayCarbon", "linewidth": 0.1}}

    # Add the molecules to the views.
    view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0")

    # Set the style.
    view.setStyle({"model": 0}, style)

    # Highlight the atoms from the mapping.
    for atom0 in lig_0.getAtoms():
        p = rdmol0.GetConformer().GetAtomPosition(atom0.index())
        # p = rdmol0.GetAtomWithIdx(atom0.index())
        view.addSphere(
            {
                "center": {"x": p.x, "y": p.y, "z": p.z},
                "radius": 0.5,
                "color": "green",
                "alpha": 0.8,
            },
        )
        view.addLabel(f"{atom0.index()}", {"position": {"x": p.x, "y": p.y, "z": p.z}})

        # Set background colour.
        view.setBackgroundColor("white")

        # Zoom to molecule.
        view.zoomTo()

    return view


def view_mapping(lig_0, lig_1, mapping):
    # Write the molecules to PDB format.
    BSS.IO.saveMolecules("molecule0", lig_0, "PDB")
    BSS.IO.saveMolecules("molecule1", lig_1, "PDB")

    # Load the molecules into RDKit.
    rdmol0 = _Chem.MolFromPDBFile("molecule0.pdb", sanitize=False, removeHs=False)
    rdmol1 = _Chem.MolFromPDBFile("molecule1.pdb", sanitize=False, removeHs=False)

    # mapping = BSS.Align.matchAtoms(
    #                         lig_0, lig_1
    #                         )

    # Set grid view properties.
    viewer0 = (0, 0)
    pixels = 900
    viewergrid = (2, 1)
    viewer1 = (1, 0)
    width = pixels / 2
    height = pixels

    # Create the view.
    view = _py3Dmol.view(
        linked=False, width=width, height=height, viewergrid=viewergrid
    )
    style = {"stick": {"colorscheme": "grayCarbon", "linewidth": 0.1}}

    # Add the molecules to the views.
    view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0", viewer=viewer0)
    view.addModel(_Chem.MolToMolBlock(rdmol1), "mol1", viewer=viewer1)

    # Set the style.
    view.setStyle({"model": 0}, style, viewer=viewer0)
    view.setStyle({"model": 0}, style, viewer=viewer1)

    # Highlight the atoms from the mapping.
    for atom0, atom1 in mapping.items():
        p = rdmol0.GetConformer().GetAtomPosition(atom0)
        view.addSphere(
            {
                "center": {"x": p.x, "y": p.y, "z": p.z},
                "radius": 0.5,
                "color": "green",
                "alpha": 0.8,
            },
            viewer=viewer0,
        )
        view.addLabel(
            f"{atom0} \u2192 {atom1}",
            {"position": {"x": p.x, "y": p.y, "z": p.z}},
            viewer=viewer0,
        )
        p = rdmol1.GetConformer().GetAtomPosition(atom1)
        view.addSphere(
            {
                "center": {"x": p.x, "y": p.y, "z": p.z},
                "radius": 0.5,
                "color": "green",
                "alpha": 0.8,
            },
            viewer=viewer1,
        )
        view.addLabel(
            f"{atom1} \u2192 {atom0}",
            {"position": {"x": p.x, "y": p.y, "z": p.z}},
            viewer=viewer1,
        )

    # Set background colour.
    view.setBackgroundColor("white", viewer=viewer0)
    view.setBackgroundColor("white", viewer=viewer1)

    # Zoom to molecule.
    view.zoomTo(viewer=viewer0)
    view.zoomTo(viewer=viewer1)

    return view

In [11]:
lig1 = "lig_17"
lig2 = "lig_6"
prot = "cmet"
prep_dir = f"/home/anna/Documents/placement_test/{prot}/prep"
name = "sys"

system_1 = BSS.IO.readMolecules(
    [
        f"{prep_dir}/{lig1}_{name}_equil_solv.rst7",
        f"{prep_dir}/{lig1}_{name}_equil_solv.prm7",
    ]
)
system_2 = BSS.IO.readMolecules(
    [
        f"{prep_dir}/{lig2}_{name}_equil_solv.rst7",
        f"{prep_dir}/{lig2}_{name}_equil_solv.prm7",
    ]
)

lig_0 = merge.extract_ligand(system_1)
lig_1 = merge.extract_ligand(system_2)

kwargs = {
    "prematch": {8:10},  # 30:27, 34:31, 41:38, 4:4
    "allow ring size change": False,
    "allow ring breaking": False,
    "complete rings only": True,
    "ring matches ring only": True,
    "match chiral tag": False,
    "natch valence": False,
    "maximise bonds": False,
    "PRUNEPERTURBEDCONSTRAINTS": True,
    "prune crossing constraints": False,
}

In [12]:
merge.no_perturbing_atoms_average(system_1, system_2, **kwargs)

15.0

In [8]:
mapping = {1:17, 2:16, 3:15, 4:14, 5:28, 6:27, 7:12, 8:11, 9:10, 10:9, 11:8, 12:7, 13:6, 14:5, 15:4, 16:3, 17:2, 18:1, 19:0}

In [13]:
l0a, l1a, mapping = pipeline.prep.merge.atom_mappings(lig_0, lig_1, **kwargs)

In [14]:
view = view_mapping(lig_0, lig_1, mapping)
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
merge_molecule = merge.merge_ligands(lig_0, lig_1, **kwargs)

BSS.Notebook.View(merge_molecule).system()

/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp4tc202c7'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpw_8pyvkx'>
  _warnings.warn(warn_message, ResourceWarning)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp_2dls3uz'>
  _warnings.warn(warn_message, ResourceWarning)
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f3532b3b760>


mapping ligands...
mapping atoms using rmsd_align, complete rings True, prune perturbed constraints True, prune crossing constraints True and prematch {}
aligning and merging the ligands according to align to LIG0, allow ring breaking False, allow ring size change False...


ThemeManager()

NGLWidget(gui_style='ngl')

In [12]:
# changes to mapping

del mapping[29]
# mapping[37] = 13

In [15]:
print(mapping)

with open(
    f"/home/anna/Documents/placement_test/cmet/inputs/mapping/{lig1}~{lig2}_mapping_dict.pickle",
    "wb",
) as handle:
    pickle.dump(mapping, handle)

{0: 18, 1: 17, 2: 16, 3: 15, 4: 14, 6: 12, 7: 11, 8: 10, 9: 9, 10: 8, 11: 7, 12: 6, 13: 5, 14: 4, 15: 3, 16: 2, 17: 1, 18: 0, 19: 29, 20: 30, 21: 31, 23: 32, 24: 33, 25: 35, 26: 34, 27: 13, 31: 45, 32: 43, 33: 42, 34: 41, 35: 39, 36: 40, 37: 38, 38: 37, 39: 59, 40: 61, 41: 62, 42: 44}


In [ ]:
view = draw_labelled_molecule(lig_0)
view

In [ ]:
view = draw_labelled_molecule(lig_1)
view